# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.20

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [3]:
def print_model_weights(model):

    for layer in model.layers:
        weights = layer.get_weights() # list of numpy arrays
        print(weights)

# Create the Data Handler

In [4]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [5]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [6]:
def get_compiled_model(shape, model_type='ann'):

    #To test the model without randomness
    #seed(1)
    
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN_5(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

In [7]:
num_features = len(selected_features)

shape = len(selected_features*window_size)  
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)
#tModel.data_handler.data_scaler = min_max_scaler
tModel.data_scaler = min_max_scaler

# Load Data

In [8]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 128

In [9]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[128. 128. 128. 128. 128.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...  0.09090909  0.24409449
   0.07882403]
 [-0

In [ ]:
#Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = min_max_scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

In [ ]:
#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = None

# Train the model and test some Tunable Model functionalities

In [10]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = len(selected_features*window_size)

#modelRULSN = RULmodel_SN_5(shape)
#modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')
#tModel.change_model('ModelRUL_SN_1', modelRULSN, 'keras')

#tModel.print_data()

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel.change_model('ModelRUL_RNN_1', model, 'keras')

#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

#print("Printing model weights")
#print_model_weights(tModel.model)

tModel.epochs = 100
tModel.train_model(learningRate_scheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 0s 25us/step - loss: 7503.2583 - mean_squared_error: 7382.9061
Epoch 2/100
17731/17731 [==============================] - 0s 5us/step - loss: 3491.8261 - mean_squared_error: 3363.3537
Epoch 3/100
17731/17731 [==============================] - 0s 5us/step - loss: 1082.6369 - mean_squared_error: 946.3630
Epoch 4/100
17731/17731 [==============================] - 0s 5us/step - loss: 701.3790 - mean_squared_error: 567.7356
Epoch 5/100
17731/17731 [==============================] - 0s 5us/step - loss: 593.4524 - mean_squared_error: 456.0304
Epoch 6/100
17731/17731 [==============================] - 0s 5us/step - loss: 535.4930 - mean_squared_error: 395.1564
Epoch 7/100
17731/17731 [==============================] - 0s 5us/step - loss: 498.9521 - mean_squared_error: 357.1321
Epoch 8/100
17731/17731 [==============================] - 0s 5us/step - loss: 465.8826 - mean_squared_error: 321.5450
Epoch 9/100
17731/17731 [=================

17731/17731 [==============================] - 0s 5us/step - loss: 255.2166 - mean_squared_error: 180.7038
Epoch 70/100
17731/17731 [==============================] - 0s 5us/step - loss: 254.2349 - mean_squared_error: 180.1309
Epoch 71/100
17731/17731 [==============================] - 0s 5us/step - loss: 254.9998 - mean_squared_error: 181.1996
Epoch 72/100
17731/17731 [==============================] - 0s 5us/step - loss: 252.5818 - mean_squared_error: 179.1979
Epoch 73/100
17731/17731 [==============================] - 0s 5us/step - loss: 252.0389 - mean_squared_error: 178.9938
Epoch 74/100
17731/17731 [==============================] - 0s 5us/step - loss: 251.6479 - mean_squared_error: 178.9308
Epoch 75/100
17731/17731 [==============================] - 0s 5us/step - loss: 252.2708 - mean_squared_error: 179.9066
Epoch 76/100
17731/17731 [==============================] - 0s 5us/step - loss: 250.3482 - mean_squared_error: 178.3136
Epoch 77/100
17731/17731 [===========================

In [11]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 538us/step
scores
Engine 1, Predicted RUL [119.256004], Rounded RUL 119.0, Real RUL 112.0
Engine 2, Predicted RUL [128.01137], Rounded RUL 128.0, Real RUL 98.0
Engine 3, Predicted RUL [51.981968], Rounded RUL 51.0, Real RUL 69.0
Engine 4, Predicted RUL [82.420525], Rounded RUL 82.0, Real RUL 82.0
Engine 5, Predicted RUL [103.510155], Rounded RUL 103.0, Real RUL 91.0
Engine 6, Predicted RUL [110.98024], Rounded RUL 110.0, Real RUL 93.0
Engine 7, Predicted RUL [109.309074], Rounded RUL 109.0, Real RUL 91.0
Engine 8, Predicted RUL [97.640274], Rounded RUL 97.0, Real RUL 95.0
Engine 9, Predicted RUL [119.464935], Rounded RUL 119.0, Real RUL 111.0
Engine 10, Predicted RUL [93.757645], Rounded RUL 93.0, Real RUL 96.0
Engine 11, Predicted RUL [84.0538], Rounded RUL 84.0, Real RUL 97.0
Engine 12, Predicted RUL [105.010056], Rounded RUL 105.0, Real RUL 124.0
Engine 13, Predicted RUL [77.254395], Rounded RUL 77.0, Real RUL 95.0
Engine 14, Predicted R